In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

def create_skin_type_model(num_classes):
    base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output_layer)
    return model

In [2]:
from tensorflow.keras.applications import InceptionV3

def create_acne_skin_tone_model(num_classes):
    base_model = InceptionV3(weights=None, include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output_layer)
    return model

In [3]:
# Create models
num_classes_skin_type = 3  # Adjust based on your data
num_classes_acne = 3       # Adjust based on your data
num_classes_skin_tone = 4  # Adjust based on your data

model_skin_type = create_skin_type_model(num_classes_skin_type)
model_acne = create_acne_skin_tone_model(num_classes_acne)
model_skin_tone = create_acne_skin_tone_model(num_classes_skin_tone)

# Load weights
try:
    model_skin_type.load_weights('mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5', by_name=True, skip_mismatch=True)
    model_acne.load_weights('inception_v3_weights_notop.h5', by_name=True, skip_mismatch=True)
    model_skin_tone.load_weights('inception_v3_weights_notop.h5', by_name=True, skip_mismatch=True)
    print("Weights loaded successfully!")
except KeyError as e:
    print(f"Error loading weights: {e}")


Weights loaded successfully!


/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_api.py:267: UserWarning: Skipping loading weights for layer #1 (named conv2d_1) due to mismatch in shape for weight conv2d_1/kernel. Weight expects shape (3, 3, 32, 32). Received saved weight with shape (3, 3, 3, 32)
  legacy_h5_format.load_weights_from_hdf5_group_by_name(
/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_api.py:267: UserWarning: Skipping loading weights for layer #4 (named conv2d_2) due to mismatch in shape for weight conv2d_2/kernel. Weight expects shape (3, 3, 32, 64). Received saved weight with shape (3, 3, 32, 32)
  legacy_h5_format.load_weights_from_hdf5_group_by_name(
/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_api.py:267: UserWarning: Skipping loading weights for layer #5 (named batch_normalization_2) due to mismatch in shape for weight batch_normalization_2/beta. Weight expects shape (64,). Received saved weight with shape (32,)
  legacy_h5_format.load

In [4]:
from tensorflow.keras.layers import concatenate

def create_combined_model():
    input_layer = model_skin_type.input
    
    # Define model outputs
    skin_type_output = model_skin_type(input_layer)
    acne_output = model_acne(input_layer)
    skin_tone_output = model_skin_tone(input_layer)
    
    # Combine outputs
    combined_output = concatenate([skin_type_output, acne_output,skin_tone_output])
    
    # Create combined model
    combined_model = Model(inputs=input_layer, outputs=combined_output)
    return combined_model

combined_model = create_combined_model()

In [5]:
combined_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 3)         │ 18,315,331 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 3)         │ 34,911,011 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 4)         │ 34,911,268 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 10)        │          0 │ functional[0][0], │
│ (Concatenate)       │                   │            │ functional_1[0][… │
│                     │                   │            │ functional_2[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 88,137,610 (336.22 MB)

 Trainable params: 88,034,634 (335.83 MB)

 Non-trainable params: 102,976 (402.25 KB)

In [6]:
combined_model.save('model.keras')

In [24]:
from tensorflow.keras.models import load_model
# Load the combined model
model = load_model('model.keras')

In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess_image(img_path):
    # Load the image with the required target size
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Normalize or preprocess as required by your model
    img_array /= 255.0  # Example normalization, adjust based on your model's needs
    
    return img_array

# Example usage
img_path = '//Users//arushreemishra//Downloads//pic.jpg.jpg'
processed_img = load_and_preprocess_image(img_path)

In [26]:
# Predict using the combined model
predictions = model.predict(processed_img)

# Assuming your model outputs are in a specific order, you may need to interpret the predictions
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[[0.33333334 0.33333334 0.33333334 0.         1.         0.
  0.24989206 0.24991009 0.25007322 0.25012463]]


In [27]:
# Example class labels
skin_type_labels = ['dry', 'normal', 'oily']  # Replace with actual skin types
acne_type_labels = ['Level_0', 'Level_1', 'Level_2']  # Replace with actual acne types
skin_tone_labels = ['mid-dark', 'light', 'dark', 'mid-light']  # Replace with actual skin tones

In [28]:
def convert_predictions_to_labels(predictions, skin_type_labels, acne_type_labels, skin_tone_labels):
    # Ensure predictions are in the correct shape
    if predictions.shape[1] != len(skin_type_labels) + len(acne_type_labels) + len(skin_tone_labels):
        raise ValueError("Predictions shape does not match the number of labels.")

    # Split predictions for each category
    skin_type_probs = predictions[:, :len(skin_type_labels)]
    acne_type_probs = predictions[:, len(skin_type_labels):len(skin_type_labels) + len(acne_type_labels)]
    skin_tone_probs = predictions[:, len(skin_type_labels) + len(acne_type_labels):]

    # Convert predictions to class labels
    skin_type_index = np.argmax(skin_type_probs[0])
    acne_type_index = np.argmax(acne_type_probs[0])
    skin_tone_index = np.argmax(skin_tone_probs[0])

    # Map indices to labels
    skin_type_label = skin_type_labels[skin_type_index]
    acne_type_label = acne_type_labels[acne_type_index]
    skin_tone_label = skin_tone_labels[skin_tone_index]

    return skin_type_label, acne_type_label, skin_tone_label

In [29]:
skin_type_label, acne_type_label, skin_tone_label = convert_predictions_to_labels(
    predictions, 
    skin_type_labels, 
    acne_type_labels, 
    skin_tone_labels
)

print(f"Skin Type: {skin_type_label}")
print(f"Acne Type: {acne_type_label}")
print(f"Skin Tone: {skin_tone_label}")

Skin Type: dry
Acne Type: Level_1
Skin Tone: mid-light
